In [1]:
import os
import re
import csv
import codecs
import pandas as pd
from datetime import datetime

In [2]:
def to_date(date_reference):
    """
    Funcao que recebe o ano (YYYY), ano e mes (YYYYmm) ou ano, mes e dia (YYYYmmdd)
    no tipo string e o transforma no tipo date.
    
    Args:
        date_reference(str): Ano, ano e mes ou ano, mes e dia.
            Ex.: '2011' ou '201105' ou '20110526'
    
    Returns:
        date: Data de referencia no formato YYYY-mm-dd.
            Ex.: '2011-05-26'
    """
    
    # Define o ano, mes e dia de referencia do arquivo
    if len(date_reference) == 4:
        year = date_reference[:4]
        month = '01'
        day = '01'
    elif len(date_reference) == 6:
        year = date_reference[:4]
        month = date_reference[4:6]
        day = '01'
    elif len(date_reference) == 8:
        year = date_reference[:4]
        month = date_reference[4:6]
        day = date_reference[6:]
    
    # Define a data de referencia do arquivo
    date = pd.to_datetime(year+'-'+month+'-'+day)
    
    return date

In [3]:
def file_references(data_file):
    """
    Funcao que recebe o nome do arquivo original e extrai o assunto e
    data de referencia.
    
    Args:
        data_file(str): Nome do arquivo original.
            Ex.: 'CNPJ_K3241.K03200DV.D00422.L00001'
    
    Returns:
        string: Data de referencia do arquivo.
            Ex.: '2020-01-01'
        string: Assunto de referencia do arquivo.
            Ex.: 'CNPJ'
    """
    # Expressao Regular para:
    # Identificar o assunto no nome do arquivo
    file_reference_re = re.compile(r'(.+)_(.+)')
    
    # Define o assunto do arquivo
    subject = re.search(file_reference_re, data_file).group(1)
    
    # Define o data de referencia do arquivo
    # Obs.: O arquivo da receita nao fornece esta informacao
    date_reference = datetime.today().strftime("%Y-%m-%d")
    
    return date_reference, subject    

In [4]:
def clean_string(text):
    """
    Funcao que recebe um texto (string) e o padroniza.
    Os passos são:
    1. Insere o termo SIGILOSO nos registros protegidos por sigilo.
    2. Insere o termo INDISPONIVEL nos registros com detalhamento nao disponivel.
    3. Remove espacos brancos extras no final dos registros.
    
    Args:
        text(str): Texto de um registro do arquivo original.
            Ex.: 'Texto com espacos extras    '
    
    Returns:
        str: Texto padronizado.
            Ex.: 'Texto com espacos extras'
    """
    # Expressao regular que identifica campos protegidos por sigilo
    confidential_re = re.compile(r'protegidas por sigilo')
    unavailable_re = re.compile(r'Detalhamento das informa')
    
    if pd.isnull(text):
        return text
    else:
        c = confidential_re.search(text)
        i = unavailable_re.search(text)
        if c:
            return u'Sigiloso'
        elif i:
            return u'Indisponivel'
        else:
            return text.strip()

In [5]:
def clean_float(text):
    """
    Funcao que recebe um campo de valor numerico com duas casas decimais
    no tipo string e o tranforma no tipo float.
    Os passos sao:
    1. Remove qualquer caractere nao numerico;
    2. Transforma no tipo de dado float;
    3. Divide por 100 para separar as casas decimais;
    
    Args:
        text(str): Valor numerico em tipo string.
            Ex.: '1.500,70'

    Returns:
        float: Valor numerico em tipo float.
            Ex.: 1500.70
    """
    # Expressao regular que identifica caracteres não numericos
    only_number_re = re.compile(r'\D')
    
    if pd.isnull(text):
        return text
    elif re.sub(only_number_re, '', text) == '':
        return ''
    else:
        return float(re.sub(only_number_re, '', text))/100

In [6]:
def clean_date(text, date_reference):
    """
    Funcao que recebe um campo de data no tipo string no formato YYYYmmdd
    e o tranforma no tipo date. Se nao for um campo data, é retornada a data
    de referencia do arquivo (para casos de campos sigilosos).
    
    Args:
        text(str): Data em tipo string no formato YYYYmmdd.
            Ex.: '19660803'
        date_reference(date): Data de referencia do arquivo origial.
            Ex.: '2020-01-01'
    Returns:
        date: Data em tipo date no formato YYYY-mm-dd.
            Ex.: '1966-08-03'
    """
    # Expressao regular que identifica data no padrão YYYYmmdd
    only_date_re = re.compile(r'([0-9]{4}[0-9]{2}[0-9]{2})')
    
    d = only_date_re.search(str(text))
    
    if d:    
        if int(text) <= 16010101 or int(text) >= 21000101:
            return ''
        else:
            return pd.to_datetime(text, format='%Y%m%d')
    else:
        return ''

In [7]:
def describe_cd_qualificacao(text):
    """
    Funcao que recebe o codigo da qualificacao do socio e retorna
    a sua descricao.
    
    Args:
        text(str): Codigo da qualificacao do socio.
            Ex.: '54'
    
    Returns:
        str: Descricao da qualificacao do socio.
            Ex.: 'Fundador'
    """
    # Dominio de qualificacao do socio
    domain = {'05': 'Administrador'
             ,'08': 'Conselheiro de Administração'
             ,'10': 'Diretor'
             ,'16': 'Presidente'
             ,'17': 'Procurador'
             ,'20': 'Sociedade Consorciada'
             ,'21': 'Sociedade Filiada'
             ,'22': 'Sócio'
             ,'23': 'Sócio Capitalista'
             ,'24': 'Sócio Comanditado'
             ,'25': 'Sócio Comanditário'
             ,'26': 'Sócio de Indústria'
             ,'28': 'Sócio-Gerente'
             ,'29': 'Sócio Incapaz ou Relat.Incapaz (exceto menor)'
             ,'30': 'Sócio Menor (Assistido/Representado)'
             ,'31': 'Sócio Ostensivo'
             ,'37': 'Sócio Pessoa Jurídica Domiciliado no Exterior'
             ,'38': 'Sócio Pessoa Física Residente no Exterior'
             ,'47': 'Sócio Pessoa Física Residente no Brasil'
             ,'48': 'Sócio Pessoa Jurídica Domiciliado no Brasil'
             ,'49': 'Sócio-Administrador'
             ,'52': 'Sócio com Capital'
             ,'53': 'Sócio sem Capital'
             ,'54': 'Fundador'
             ,'55': 'Sócio Comanditado Residente no Exterior'
             ,'56': 'Sócio Comanditário Pessoa Física Residente no Exterior'
             ,'57': 'Sócio Comanditário Pessoa Jurídica Domiciliado no Exterior'
             ,'58': 'Sócio Comanditário Incapaz'
             ,'59': 'Produtor Rural'
             ,'63': 'Cotas em Tesouraria'
             ,'65': 'Titular Pessoa Física Residente ou Domiciliado no Brasil'
             ,'66': 'Titular Pessoa Física Residente ou Domiciliado no Exterior'
             ,'67': 'Titular Pessoa Física Incapaz ou Relativamente Incapaz (exceto menor)'
             ,'68': 'Titular Pessoa Física Menor (Assistido/Representado)'
             ,'70': 'Administrador Residente ou Domiciliado no Exterior'
             ,'71': 'Conselheiro de Administração Residente ou Domiciliado no Exterior'
             ,'72': 'Diretor Residente ou Domiciliado no Exterior'
             ,'73': 'Presidente Residente ou Domiciliado no Exterior'
             ,'74': 'Sócio-Administrador Residente ou Domiciliado no Exterior'
             ,'75': 'Fundador Residente ou Domiciliado no Exterior'}
    
    if text in domain.keys():
        return domain[text]
    else:
        return 'Descrição do código não informada.'

In [8]:
def describe_in_cpf_cnpj(text):
    """
    Funcao que recebe o codigo do indicador de CPF/CNPJ
    e retorna sua descricao.
    
    Args:
        text(str): Codigo do indicador de CPF/CNPJ.
            Ex.: '2'
    
    Returns:
        str: Descricao do indicador de CPF/CNPJ.
            Ex.: 'Pessoa Fisica'
    """
    # Dominio de qualificacao do socio
    domain = {'1': 'Pessoa Jurídica'
             ,'2': 'Pessoa Física'
             ,'3': 'Nome Exterior'}
    
    if text in domain.keys():
        return domain[text]
    else:
        return 'Descrição do código não informada.'

In [9]:
def load_data_frame(in_fields, in_data_path):
    """
    Funcao que recebe o caminho completo de um arquivo de dados ja codificado
    em UTF-8, prepara-o em um data frame e renomeia as colunas.
    
    Args:
        in_fields(array): Lista com os nomes das colunas para o data frame.
            Ex.: ['cd_campo', 'nm_campo', 'dt_campo', 'vl_campo']
        in_data_path(str): Caminho completo de acesso ao arquivo de dados.
            Ex.: '..\\data\\encoded\\201301_GastosDiretos.csv'
        
    Returns:
        dataframe: Data frame do arquivo original com as colunas renomeadas.
    """
    # Le o arquivo original em um dataframe
    df = pd.read_csv(in_data_path
                    ,sep='\t'
                    #,quotechar = '\"'
                    ,quoting = csv.QUOTE_NONE
                    ,low_memory = False
                    ,encoding = 'utf-8'
                    ,dtype = 'object')
    
    #Renomeia as colunas
    df.columns = in_fields
    
    return df

In [10]:
def data_frame_to_csv(df, out_data_path):
    """
    Funcao que recebe o caminho completo para escrita do dataframe e o
    transforma em arquivo CSV, codificado em UTF-8 e separado por tab.
    
    Args:
        df(dataframe): Dataframe com os campos padronizados.
        out_data_path(str): Caminho completo para escrita do dataframe em csv.
            Ex.: '..\\data\\padronized\\Favorecidos_GastosDiretos_2013-01-01.csv'
    """
    df.to_csv(path_or_buf = out_data_path
             ,index = False
             ,sep = '\t'
             ,encoding = 'utf-8')

In [11]:
def select_fields(df, fields_list):
    """
    Funcao que recebe um data frame e retorna uma copia com apenas as
    colunas informadas.
    
    Args:
        df(dataframe): Data frame enviado para recorte.
        fields_list(array): Lista de colunas do dataframe para selecao.
        
    Returns:
        dataframe: Copia do dataframe apenas com as colunas informadas.
    """
    return df.loc[:,fields_list].copy()

In [12]:
def add_data_source_fields(df, **kwargs):
    """
    Funcao que adiciona ao final do dataframe uma sequencia de campos
    referentes a fonte dos dados que o originou.
    
    Args:
        df(dataframe): Dataframe com os campos padronizados.
        date_reference(date): Data de referencia do arquivo original.
            Ex.: '2013-01-01'
        data_source(str): Nome da fonte do arquivo original.
            Ex.: 'Portal da Transparência'
        data_file(str): Nome do arquivo original.
            Ex,: 'GastosDiretos_201301.csv'
        
    Returns:
        dataframe: Dataframe com os campos referentes a 
            fonte de dados adicionado.
    """
    df['dt_referencia'] = kwargs['date_reference']
    df['nm_fonte_dados'] = kwargs['data_source']
    df['nm_arquivo_dados'] = kwargs['data_file'] 
    
    return df

In [13]:
def standardize_cnpj(**kwargs):
    """
    Recebe a localizacao fisica do arquivo organizado, renomeia as colunas, padroniza os tipos 
    de dados (texto, numerico e data) e cria um novo arquivo padronizado no local informado.
    
    Args:
        data_file(str): Nome do arquivo original.
            Ex.: 'CNPJ_K3241.K03200DV.D00422.L00001'
        date_reference(date): Data de referencia do arquivo original.
            Ex.: '2010-01-01'
        in_data_path(str): Caminho completo para acesso ao arquivo original.
            Ex.: '..\data\03-organized\CNPJ_K3241.K03200DV.D00422.L00001'
        out_data_path(str): Caminho completo onde deve ser criado o(s) novo(s)
            arquivo(s) com os dados padronizados.
            Ex.: '..\data\04-standardized\CNPJ_K3241.K03200DV.D00422.L00001.csv'
    """
        
    # Nome das colunas que irao substituir o nome das colunas originais
    kwargs['in_fields'] =  ['tp_registro'
                            ,'in_forma_envio'
                            ,'tp_atualizacao'
                            ,'nr_cnpj'
                            ,'in_matriz_filial'
                            ,'nm_razao_social'
                            ,'nm_fantasia'
                            ,'in_situacao_cadastral'
                            ,'dt_situacao_cadastral'
                            ,'cd_motivo_situacao_cadastral'
                            ,'nm_cidade_exterior'
                            ,'co_pais'
                            ,'nm_pais'
                            ,'cd_natureza_juridica'
                            ,'dt_inicio_atividade'
                            ,'cd_cnae_principal'
                            ,'tp_logradouro'
                            ,'ds_logradouro'
                            ,'nr_logradouro'
                            ,'ds_logradouro_complemento'
                            ,'ds_logradouro_bairro'
                            ,'nr_cep'
                            ,'sg_uf'
                            ,'cd_municipio'
                            ,'nm_municipio'
                            ,'nr_telefone_ddd_01'
                            ,'nr_telefone_001'
                            ,'nr_telefone_ddd_02'
                            ,'nr_telefone_002'
                            ,'nr_fax_ddd'
                            ,'nr_fax'
                            ,'ds_correio_eletronico'
                            ,'cd_qualificacao_responsalvel'
                            ,'vl_capital_social'
                            ,'cd_porte_empresa'
                            ,'cd_opcao_simples'
                            ,'dt_opcao_simples_inclusao'
                            ,'dt_opcao_simples_exclusao'
                            ,'in_existe_opcao_mei'
                            ,'ds_situacao_especial'
                            ,'dt_situacao_especial']
    
    # Prepara o CSV original em um dataframe e renomeia as colunas 
    df = load_data_frame(kwargs['in_fields'], kwargs['in_data_path'])
    
    # Padroniza todos os campos do dataframe
    df = df.applymap(lambda x: clean_string(x))
    
    # Padroniza os campos de data e numericos
    df['dt_situacao_cadastral'] = df['dt_situacao_cadastral'].apply(lambda x: clean_date(x, kwargs['date_reference']))
    df['dt_inicio_atividade'] = df['dt_inicio_atividade'].apply(lambda x: clean_date(x, kwargs['date_reference']))
    df['dt_opcao_simples_inclusao'] = df['dt_opcao_simples_inclusao'].apply(lambda x: clean_date(x, kwargs['date_reference']))
    df['dt_opcao_simples_exclusao'] = df['dt_opcao_simples_exclusao'].apply(lambda x: clean_date(x, kwargs['date_reference']))
    df['dt_situacao_especial'] = df['dt_situacao_especial'].apply(lambda x: clean_date(x, kwargs['date_reference']))    
    df['vl_capital_social'] = df['vl_capital_social'].apply(clean_float)
    
    # Inclui os campos de informacao da fonte dos dados
    df = add_data_source_fields(df, **kwargs)
    
    # Nome das colunas que irao substituir o nome das colunas originais
    kwargs['out_fields'] = ['tp_registro'
                            ,'in_forma_envio'
                            ,'tp_atualizacao'
                            ,'nr_cnpj'
                            ,'in_matriz_filial'
                            ,'nm_razao_social'
                            ,'nm_fantasia'
                            ,'in_situacao_cadastral'
                            ,'dt_situacao_cadastral'
                            ,'cd_motivo_situacao_cadastral'
                            ,'nm_cidade_exterior'
                            ,'co_pais'
                            ,'nm_pais'
                            ,'cd_natureza_juridica'
                            ,'dt_inicio_atividade'
                            ,'cd_cnae_principal'
                            ,'tp_logradouro'
                            ,'ds_logradouro'
                            ,'nr_logradouro'
                            ,'ds_logradouro_complemento'
                            ,'ds_logradouro_bairro'
                            ,'nr_cep'
                            ,'sg_uf'
                            ,'cd_municipio'
                            ,'nm_municipio'
                            ,'nr_telefone_ddd_01'
                            ,'nr_telefone_001'
                            ,'nr_telefone_ddd_02'
                            ,'nr_telefone_002'
                            ,'nr_fax_ddd'
                            ,'nr_fax'
                            ,'ds_correio_eletronico'
                            ,'cd_qualificacao_responsalvel'
                            ,'vl_capital_social'
                            ,'cd_porte_empresa'
                            ,'cd_opcao_simples'
                            ,'dt_opcao_simples_inclusao'
                            ,'dt_opcao_simples_exclusao'
                            ,'in_existe_opcao_mei'
                            ,'ds_situacao_especial'
                            ,'dt_situacao_especial'
                            ,'dt_referencia'
                            ,'nm_fonte_dados'
                            ,'nm_arquivo_dados']
    
    # Prepara um data frame com apenas as colunas do arquivo de saida
    sub_df = select_fields(df, kwargs['out_fields'])
    
    # Salva o arquivo de saida no local informado
    data_frame_to_csv(sub_df, kwargs['out_data_path'])

    # Log: Mensagem de fim do processo
    print(str(datetime.now()) + ': Arquivo ' + kwargs['data_file'] + ' padronizado.')

In [14]:
def standardize_socio(**kwargs):
    """
    Recebe a localizacao fisica do arquivo organizado, renomeia as colunas, padroniza os tipos 
    de dados (texto, numerico e data) e cria um novo arquivo padronizado no local informado.
    
    Args:
        data_file(str): Nome do arquivo original.
            Ex.: 'SOCIO_K3241.K03200DV.D00422.L00001'
        date_reference(date): Data de referencia do arquivo original.
            Ex.: '2010-01-01'
        in_data_path(str): Caminho completo para acesso ao arquivo original.
            Ex.: '..\data\03-organized\SOCIO_K3241.K03200DV.D00422.L00001'
        out_data_path(str): Caminho completo onde deve ser criado o(s) novo(s)
            arquivo(s) com os dados padronizados.
            Ex.: '..\data\04-standardized\SOCIO_K3241.K03200DV.D00422.L00001.csv'
    """
        
    # Nome das colunas que irao substituir o nome das colunas originais
    kwargs['in_fields'] =  [ 'tp_registro'
                            ,'in_forma_envio'
                            ,'tp_atualizacao'
                            ,'nr_cnpj'
                            ,'tp_socio'
                            ,'nm_socio'
                            ,'nr_cpf_cnpj_socio'
                            ,'cd_qualificacao_socio'
                            ,'pc_capital_social'
                            ,'dt_sociedade_inclusao'
                            ,'cd_pais_socio'
                            ,'nm_pais_socio'
                            ,'nr_cpf_representante_legal'
                            ,'nm_representante_legal'
                            ,'cd_qualificacao_rep_legal']
    
    # Prepara o CSV original em um dataframe e renomeia as colunas 
    df = load_data_frame(kwargs['in_fields'], kwargs['in_data_path'])
    
    # Padroniza todos os campos do dataframe
    df = df.applymap(lambda x: clean_string(x))
    
    # Padroniza os campos de data e numericos
    df['dt_sociedade_inclusao'] = df['dt_sociedade_inclusao'].apply(lambda x: clean_date(x, kwargs['date_reference']))       
    df['pc_capital_social'] = df['pc_capital_social'].apply(clean_float)
    
    # Inclui os campos de informacao da fonte dos dados
    df = add_data_source_fields(df, **kwargs)
    
    # Nome das colunas que irao substituir o nome das colunas originais
    kwargs['out_fields'] = [ 'tp_registro'
                            ,'in_forma_envio'
                            ,'tp_atualizacao'
                            ,'nr_cnpj'
                            ,'tp_socio'
                            ,'nm_socio'
                            ,'nr_cpf_cnpj_socio'
                            ,'cd_qualificacao_socio'
                            ,'pc_capital_social'
                            ,'dt_sociedade_inclusao'
                            ,'cd_pais_socio'
                            ,'nm_pais_socio'
                            ,'nr_cpf_representante_legal'
                            ,'nm_representante_legal'
                            ,'cd_qualificacao_rep_legal'
                            ,'dt_referencia'
                            ,'nm_fonte_dados'
                            ,'nm_arquivo_dados']
    
    # Prepara um data frame com apenas as colunas do arquivo de saida
    sub_df = select_fields(df, kwargs['out_fields'])
    
    # Salva o arquivo de saida no local informado
    data_frame_to_csv(sub_df, kwargs['out_data_path'])

    # Log: Mensagem de fim do processo
    print(str(datetime.now()) + ': Arquivo ' + kwargs['data_file'] + ' padronizado.')

In [15]:
def standardize_cnae(**kwargs):
    """
    Recebe a localizacao fisica do arquivo organizado, renomeia as colunas, padroniza os tipos 
    de dados (texto, numerico e data) e cria um novo arquivo padronizado no local informado.
    
    Args:
        data_file(str): Nome do arquivo original.
            Ex.: 'SOCIO_K3241.K03200DV.D00422.L00001'
        date_reference(date): Data de referencia do arquivo original.
            Ex.: '2010-01-01'
        in_data_path(str): Caminho completo para acesso ao arquivo original.
            Ex.: '..\data\03-organized\SOCIO_K3241.K03200DV.D00422.L00001'
        out_data_path(str): Caminho completo onde deve ser criado o(s) novo(s)
            arquivo(s) com os dados padronizados.
            Ex.: '..\data\04-standardized\SOCIO_K3241.K03200DV.D00422.L00001.csv'
    """
        
    # Nome das colunas que irao substituir o nome das colunas originais
    kwargs['in_fields'] =  [ 'tp_registro'
                            ,'in_forma_envio'
                            ,'tp_atualizacao'
                            ,'nr_cnpj'
                            ,'cd_cnae_secundaria']
    
    # Prepara o CSV original em um dataframe e renomeia as colunas 
    df = load_data_frame(kwargs['in_fields'], kwargs['in_data_path'])
    
    # Padroniza todos os campos do dataframe
    df = df.applymap(lambda x: clean_string(x))
    
    # Inclui os campos de informacao da fonte dos dados
    df = add_data_source_fields(df, **kwargs)
    
    # Nome das colunas que irao substituir o nome das colunas originais
    kwargs['out_fields'] = [ 'tp_registro'
                            ,'in_forma_envio'
                            ,'tp_atualizacao'
                            ,'nr_cnpj'
                            ,'cd_cnae_secundaria'
                            ,'dt_referencia'
                            ,'nm_fonte_dados'
                            ,'nm_arquivo_dados']
    
    # Prepara um data frame com apenas as colunas do arquivo de saida
    sub_df = select_fields(df, kwargs['out_fields'])
    
    # Salva o arquivo de saida no local informado
    data_frame_to_csv(sub_df, kwargs['out_data_path'])

    # Log: Mensagem de fim do processo
    print(str(datetime.now()) + ': Arquivo ' + kwargs['data_file'] + ' padronizado.')

In [16]:
def standardize_files():
    """
    Acessa os arquivos da pasta 03-organized, passa pelo processo de padronizacao
    e armazena-os na pasta 04-standardized.
    """

    # Prepara o dicionario de variaveis (kwargs = keyworded arguments)
    kwargs = {}
    
    # Nome da fonte dos dados
    kwargs['data_source'] = u'Receita Federal'
    
    # Diretorio de armazenamento dos arquivos originais
    kwargs['in_data_dir'] = '..\\data\\03-organized'
    
    # Diretorio de armazenamento dos arquivos tratados
    kwargs['out_data_dir'] = '..\\data\\04-standardized'
    
    # Lista dos arquivos organizados 
    kwargs['data_files'] = os.listdir(kwargs['in_data_dir'])
    
    # Log: Mensagem de inicio do processo
    print(str(datetime.now()) + ': Padronizacao dos arquivos iniciada.')
    
    # Para cada arquivo na lista de arquivos organizados
    for file in kwargs['data_files']:
        
        # Define o nome do arquivo
        kwargs['data_file'] = file

        # Define o caminho completo de acesso ao arquivo original
        kwargs['in_data_path'] = os.path.join(kwargs['in_data_dir'], kwargs['data_file'])

        # Define o caminho completo de armazenamento do arquivo padronizado
        kwargs['out_data_path'] = os.path.join(kwargs['out_data_dir'], kwargs['data_file'])
        
        # Define data e assunto de referencia do arquivo
        kwargs['date_reference'], kwargs['subject'] = file_references(kwargs['data_file'])

        # Padroniza os arquivos
        if kwargs['subject'] == 'CNPJ':
            standardize_cnpj(**kwargs)   
        if kwargs['subject'] == 'SOCIO':
            standardize_socio(**kwargs) 
        if kwargs['subject'] == 'CNAE':
            standardize_cnae(**kwargs) 
            
      
    # Log: Mensagem de finalizacao do processo
    print(str(datetime.now()) + ': Padronizacao dos arquivos finalizada.')

In [19]:
def main():
    """
    Padroniza os dados do Cadastro Nacional de Pessoas Juridicas (CNPJ).
    Origem: Ministerio da Economia - Receita Federal
    """
    
    ### Padroniza os arquivos
    standardize_files()

if __name__ == '__main__':
    main()

2021-03-16 09:01:14.113240: Padronizacao dos arquivos iniciada.
2021-03-16 09:01:35.018306: Arquivo CNAE_K3241.K032001K.CNPJ.D01120.L00001.csv padronizado.
2021-03-16 09:02:01.559979: Arquivo CNAE_K3241.K032001K.CNPJ.D01120.L00002.csv padronizado.
2021-03-16 09:02:31.235284: Arquivo CNAE_K3241.K032001K.CNPJ.D01120.L00003.csv padronizado.
2021-03-16 09:02:58.909969: Arquivo CNAE_K3241.K032001K.CNPJ.D01120.L00004.csv padronizado.
2021-03-16 09:03:37.204730: Arquivo CNAE_K3241.K032001K.CNPJ.D01120.L00005.csv padronizado.
2021-03-16 09:16:24.962706: Arquivo CNPJ_K3241.K032001K.CNPJ.D01120.L00001.csv padronizado.
2021-03-16 09:29:01.477390: Arquivo CNPJ_K3241.K032001K.CNPJ.D01120.L00002.csv padronizado.
2021-03-16 09:41:49.908996: Arquivo CNPJ_K3241.K032001K.CNPJ.D01120.L00003.csv padronizado.
2021-03-16 09:56:40.919418: Arquivo CNPJ_K3241.K032001K.CNPJ.D01120.L00004.csv padronizado.
2021-03-16 10:09:34.908071: Arquivo CNPJ_K3241.K032001K.CNPJ.D01120.L00005.csv padronizado.
2021-03-16 10:15